# script anagrafica 02

Authors: Fabio Morea, Leyla Vesnic @ Area Science Park
 
Description: python scripts to clean and prepare data on regional companies.
This script imports data from 'IMPRESE.xlsx' and produces clean data in the form of .csv files.

License: 
scripts are available under CC-BY-4.0
data is not included in the package

Code is mantained on GitLab: https://gitlab.com/area-science-park-sustainability/imprese-fvg

## Obiettivo
L'obiettivo di questo script è convertire i risultati dallo script 01
in una serie di tabelle ulteriormente pulite e ordinate, pronte per l'utilizzo in dashboard e analisi.

## dati di input per questo script
Questo script parte dai due file generati dallo script 01:

- imprese_anagrafica.csv
- imprese_codici.csv

## risultati prodotti da questo script
I risultati sono una serie di file .csv, ciascuno corrispondente ad una tabella ottimizzata per creare
un database e adatta come punto di partenza per elaborazioni in powerBi e progetti di Machine
Learning. In particolare le tabelle di output:

1. rispettano i requisiti “tidy” [TODO: spiegare meglio]
2. aggiungono ad ogni riga due metadati: Fonte e Data di aggiornamento. in questo modo possiamo
sempre risalire all’origine del dato ed evitare di confrontare dati disomogenei. Questa
struttura è pronta per la storicizzazione: possiamo immaginare di accodare file olologhi aggiornati
in mesi diversi e ricostruire l’evoluzione nel tempo di un attributo.
3. distinguono in maniera più chiara tra “imprese” e “localizzazioni”, creando due identificativi
univoci: id_impresa (ID univoco dell’impresa) e id_localiz (ID univoco della localizzazione).
Questo punto è molto importante perchè alcune caratteristiche sono proprie dell’impresa (es.
è una startup) e altre delle localizzazioni (es. indirizzo).
4. rispettano la privacy grazie ad alcune tecniche base di pseudonimizzazione per tutelare la
privacy. Prima di tutto si suddividono i dati “potenzialmente personali” (CF, denominazione,
indirizzo…) dagli altri. Per garantire al meglio l aprovacy è sufficiente omettere le tabelle con
i dati personali e lavorare solo con le altre. Le imprese vengono identificate con con un
id_impresa e un id_localizzazione, che non contengono informazioni personali.
5. codificano i testi duplicati usando dei “dizionari” (ad esempio la descrizione estesa dei codici
ateco e la natura giuridica delle imprese si possono codificare. La tabella principale t_*
contiene il codice, mentre una seconda tabella “dizionario” (file che inizia con d_*) consente di associare codice
e descrizione estesa)
6. separano dati poco utilizzati (ad esempio la descrizione delle attività, poco utilizzata nelle
analisi, viene salvata in un file separato)
7. uniscono in una colonna unica le informazioni sparse nelle colonne “tipo_ul_sede”

**NOTA:** Alcune ulteriori elaborazioni da inserire:

- aggiungere il codice ISTAT del Comune
- creare una tabella di corrispondenza tra codice ISTAT, nome del comune e aggregazioni di
comuni come ad esempio SLL (sistemi locali del lavoro), comuni montani, UTI, zona costiera,...

## Operazioni preliminari: librerie e directory di riferimento

In [19]:
# Setup
import sys
import os
from pathlib import Path
import datetime
import pandas as pd
pd.options.display.max_columns = None

In [20]:
current_path = Path(os.getcwd())
data_subdir = "data"
data_path = current_path.parent / data_subdir
data_dir = str(data_path)

repo_subdir = "repository"
repo_path = current_path.parent / repo_subdir
repo_dir = str(repo_path)
 
repo_dir, data_dir


('c:\\Users\\longato\\OneDrive - Area Science Park\\Data Repository\\dataset_imprese_fvg\\repository',
 'c:\\Users\\longato\\OneDrive - Area Science Park\\Data Repository\\dataset_imprese_fvg\\data')

In [21]:
filename = data_dir +'\\imprese_anagrafica.csv'
col_names = pd.read_csv(filename, nrows=0,sep = '|').columns
dtypes={'id_localiz': int, 'id_impresa': int,  'piva':str}
types_dict = {'id_localiz': int, 'id_impresa': int}
types_dict.update({col: str for col in col_names if col not in types_dict})
parse_dates = [d for d in col_names if d.startswith('data')]

df_anagrafica = pd.read_csv (  filename, sep = '|', dtype=types_dict, parse_dates=parse_dates)
df_anagrafica.info()

C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2688483337.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_anagrafica = pd.read_csv (  filename, sep = '|', dtype=types_dict, parse_dates=parse_dates)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2688483337.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_anagrafica = pd.read_csv (  filename, sep = '|', dtype=types_dict, parse_dates=parse_dates)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2688483337.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_anagrafica = pd.read_csv (  filename, sep 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193527 entries, 0 to 193526
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   fonte                  193527 non-null  object        
 1   mm_aaaa                193527 non-null  object        
 2   id_impresa             193527 non-null  int32         
 3   id_localiz             193527 non-null  int32         
 4   tipo_sedeul            88280 non-null   object        
 5   denominazione          193527 non-null  object        
 6   cf                     193527 non-null  object        
 7   piva                   193176 non-null  object        
 8   reg_imp_n              192087 non-null  object        
 9   n-albo_art             31250 non-null   object        
 10  reg_imp_sez            192050 non-null  object        
 11  tipo_impresa           193527 non-null  object        
 12  ng_esteso              193527 non-null  obje

In [22]:
filename = data_dir +'\\imprese_codici.csv'
dtypes={'id_localiz': int, 'id_impresa': int, 'loc_n':int }
cols_to_use = [ 'mm_aaaa', 'fonte', 'id_localiz', 'id_impresa' ,'loc_n', 'ateco_tipo', 'ateco', 'ateco_desc'] 
df_codici = pd.read_csv (  filename, sep = '|', usecols = cols_to_use , dtype = dtypes)
df_codici.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   fonte       0 non-null      object
 1   mm_aaaa     0 non-null      object
 2   id_impresa  0 non-null      int32 
 3   id_localiz  0 non-null      int32 
 4   loc_n       0 non-null      int32 
 5   ateco_tipo  0 non-null      object
 6   ateco       0 non-null      object
 7   ateco_desc  0 non-null      object
dtypes: int32(3), object(5)
memory usage: 132.0+ bytes


# Creazione dei "dizionari" csv

In [23]:
# creo file csv per tabela imprese - localizzazioni
file_risultati  = repo_dir + '\\' + 'id_imp_loc.csv'
cols_to_use = ['id_impresa', 'id_localiz']
df = df_anagrafica[ cols_to_use ]
df.to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False)
 

In [24]:
# Crea dizionario descrizioni ateco
file_risultati  = repo_dir + '\\' +  'd_ateco.csv'
cols_to_use = ['ateco', 'ateco_desc']
df = df_codici[cols_to_use]
df.drop_duplicates(inplace = True)
df.sort_values(by='ateco', ascending=True, inplace=True)
df.to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False)

#   df_codici.drop(columns = 'ateco_desc', axis = 1) 
   

C:\Users\longato\AppData\Local\Temp\ipykernel_31420\1578593468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='ateco', ascending=True, inplace=True)


In [25]:
# Crea dizionario ng
file_risultati  = repo_dir + '\\' + 'd_ng.csv'
cols_to_use = ['tipo_impresa', 'ng2', 'ng_esteso']
df = df_anagrafica[ cols_to_use ]
df['ng_esteso'] =  df['ng_esteso'].str[5:]
df.drop_duplicates(inplace = True)
df.sort_values(by=['tipo_impresa','ng2'], ascending=False, inplace=True)

df.to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False)
#df_anagrafica.drop(columns = 'ng_esteso', axis = 1)   

C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2229218810.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ng_esteso'] =  df['ng_esteso'].str[5:]
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2229218810.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace = True)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2229218810.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [26]:
# Crea lista descrizioni attività

# NB c'è qualche problema di caratteri impropri / taglia le descrizioni 
cols_to_use = [ 'id_impresa', 'descrizione_attivita']
file_risultati  = repo_dir + '\\' + 't_attivita.csv'

df = df_anagrafica[cols_to_use]

df.drop_duplicates( inplace = True)

df.sort_values(by='id_impresa', ascending=True, inplace=True)

df.to_csv(file_risultati,sep ='|',encoding='utf-8-sig',  index=False)


C:\Users\longato\AppData\Local\Temp\ipykernel_31420\1673190287.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates( inplace = True)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\1673190287.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='id_impresa', ascending=True, inplace=True)


In [27]:
#crea il file delle localizzazioni
file_risultati  = repo_dir + '\\' + 't_localizz.csv'
cols_localiz =      [
                    'fonte',   'id_localiz', 'id_impresa', 'denominazione',
                    'tipo_localizzazione',                    
                    'data_apert_ul', 
                    'comune', 'indirizzo',
                    'tipo_sedeul'
                    ]
df = df_anagrafica[cols_localiz]
#df.fillna('', inplace=True)
df
 

,fonte,id_localiz,id_impresa,denominazione,tipo_localizzazione,data_apert_ul,comune,indirizzo,tipo_sedeul
0,I,1,1,P E L L I Z Z A R I S I L V I O D I S E ...,SE - SEDE PRINCIPALE,NaT,CORMONS - GO,V I A P E S C H E R I A 4,NaN
1,I,2,2,B . F . B . C A S A D I S P E D I Z ...,SE - SEDE PRINCIPALE,NaT,TRIESTE - TS,V I A C O R T I 2,NaN
2,I,3,2,B . F . B . C A S A D I S P E D I Z ...,UL - UNITÀ LOCALE,2007-08-01,MONRUPINO - TS,F E R N E T T I 5,U - UFFICIO
3,I,4,2,B . F . B . C A S A D I S P E D I Z ...,UL - UNITÀ LOCALE,2015-10-15,TRIESTE - TS,P U N T O F R A N C O N U O V O E X ...,U - UFFICIO
4,I,6,3,C A R R O Z Z E R I A A Z Z A N E S E D I ...,SE - SEDE PRINCIPALE,NaT,AZZANO DECIMO - PN,V I A F I U M I C I N O 1 4,NaN
...,...,...,...,...,...,...,...,...,...
193522,I,199370,103261,Z O Z Z O L I A N T O N I O,SE - SEDE PRINCIPALE,NaT,PAULARO - UD,V I A A O N E S I 4 2,NaN
193523,I,199371,103262,E S T E T I C A S A G I T T A R I O D I ...,SE - SEDE PRINCIPALE,NaT,TRIESTE - TS,V I A P A L E S T R I N A 3,NaN
193524,I,199372,103263,Z A Z Z A R I N I R O B E R T O,SE - SEDE PRINCIPALE,NaT,VALVASONE ARZENE - PN,V I A S A N T A M A R G H E R I T A 2 5,NaN
193525,I,199373,103264,Z O Z Z O L I S I L V A N A,SE - SEDE PRINCIPALE,NaT,PAULARO - UD,F R A Z I O N E R A V I N I S 1 9 / A,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193527 entries, 0 to 193526
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   fonte                193527 non-null  object        
 1   id_localiz           193527 non-null  int32         
 2   id_impresa           193527 non-null  int32         
 3   denominazione        193527 non-null  object        
 4   tipo_localizzazione  193527 non-null  object        
 5   data_apert_ul        88625 non-null   datetime64[ns]
 6   comune               193527 non-null  object        
 7   indirizzo            193268 non-null  object        
 8   tipo_sedeul          88280 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(6)
memory usage: 11.8+ MB


# Unificazione dei campi "tipo sede ul"

In [29]:
# tipo_localizzazione: 
# valori possibili: 
# {'ST - SEDE TRASFERITA', 'SS - SEDE SECONDARIA', 'UL - UNITÀ LOCALE', 'SE - SEDE PRINCIPALE'}
print(set (df['tipo_localizzazione']))
df['tipo_localizzazione'] = df['tipo_localizzazione'].str[0:2]

{'ST - SEDE TRASFERITA', 'SS - SEDE SECONDARIA', 'SE - SEDE PRINCIPALE', 'UL - UNITÀ LOCALE'}


C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2220071272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_localizzazione'] = df['tipo_localizzazione'].str[0:2]


In [30]:
# lista di tutti i tipi disponibili
# tipi = set(df.tipo_sedeul_1) | set(df.tipo_sedeul_2) | set(df.tipo_sedeul_3) | set(df.tipo_sedeul_4) | set(df.tipo_sedeul_5)
# lista_tipi = sorted(list(tipi) )
# print(lista_tipi)

In [31]:
#unisce i testi tipo_sedeul in unico campo
#cols_to_use = 
#df2 = df[cols_to_use]#.astype(str)

# def unione_campi(row):
    # unione = ""
    # if len(row.tipo_sedeul_1) > 0:
        # unione = unione + " > " + row.tipo_sedeul_1
    # if len(row.tipo_sedeul_2) > 0:
        # unione = unione + " > " + row.tipo_sedeul_2
    # if len(row.tipo_sedeul_3) > 0:
        # unione = unione + " > " + row.tipo_sedeul_3
    # if len(row.tipo_sedeul_4) > 0:
        # unione = unione + " > " + row.tipo_sedeul_4
    # if len(row.tipo_sedeul_5) > 0:
        # unione = unione + " > " + row.tipo_sedeul_5
    # n = unione.count("-")
    # if n > 0: 
        # unione = str(n) + ": " + unione
    # return unione

# column = df.apply(unione_campi, axis = 1)

# df = df.assign(tipo_sedeul = column.values)

# df.drop(columns = ['tipo_sedeul_1', 'tipo_sedeul_2','tipo_sedeul_3', 'tipo_sedeul_4','tipo_sedeul_5'], inplace = True)


In [32]:
# le localizzazioni possono avere più di un "tipo" 
# https://www.registroimprese.it/sede-legale-e-unita-locali
# set(df['tipo_sedeul'])
# da fare: riportare le descrizioni estese in una tabella separata

In [33]:
df.sort_values(by='id_localiz', ascending=True, inplace=True)
df['prov_localiz'] = df['comune'].str[-3:]
df['comune'] = df['comune'].str[:-5] # DA MIGLIORARE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

C:\Users\longato\AppData\Local\Temp\ipykernel_31420\1155195190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='id_localiz', ascending=True, inplace=True)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\1155195190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prov_localiz'] = df['comune'].str[-3:]
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\1155195190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [34]:

cols_order = ['fonte',  'id_localiz', 'id_impresa', 'denominazione', 'tipo_localizzazione',  'data_apert_ul', 'prov_localiz', 'comune' , 'indirizzo', 'tipo_sedeul']

df[cols_order].to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False)
df.columns

Index(['fonte', 'id_localiz', 'id_impresa', 'denominazione',
       'tipo_localizzazione', 'data_apert_ul', 'comune', 'indirizzo',
       'tipo_sedeul', 'prov_localiz'],
      dtype='object')

TODO : NB migliorare l'estrazione del Comune: se c'è "-" va bene così, se si tratta di comune estero con questa formula viene troncato 

# pseudonimizzazione con id_impresa

In [35]:
# crea file csv con pseudonimizzazione dei CF <> id_impresa
file_risultati  = repo_dir + '\\' + 'pseudo_cf_id_impresa.csv'

cols_imp =          ['fonte',  'id_impresa', 'cf']   
df = df_anagrafica[cols_imp]

df.drop_duplicates(subset = 'id_impresa', inplace = True) 
df.sort_values(by='id_impresa', ascending=True, inplace=True)

df.to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False )

C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2930013361.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset = 'id_impresa', inplace = True)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\2930013361.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='id_impresa', ascending=True, inplace=True)


In [36]:
file_risultati  = repo_dir + '\\' + 't_imprese.csv'

cols_imp =          [
                    'fonte',  'mm_aaaa', 'id_impresa',  
'cf', 'piva', 'denominazione',
                    'prov', 'reg_imp_n',  'sede_ul', 'n-albo_art',
                    'reg_imp_sez', 'ng2',   
                    'stato_impresa',
                    'data_cost','data_isc_ri', 'data_isc_rd', 'data_isc_aa',
                    'data_canc', 'data_ini_at', 
                    'data_cess_att',  'data_fall', 'data_liquid',
                    'addetti_aaaa', 'addetti_indip', 'addetti_dip', 
                    'capitale', 'capitale_valuta',
                    'imp_sedi_ee', 'imp_eefvg', 'imp_pmi', 'imp_startup',
                    'imp_femmilile', 'imp_giovanile', 'imp_straniera' , 
                    
                    ]   

df = df_anagrafica[cols_imp]

## attenzione! "drop duplicates" non va bene perchè seleziona la provincia "a caso"
#  è necessario scegliere la provincia della sede legale con n_sede = 0
df.sort_values(['cf', 'sede_ul' ], inplace = True)
df['cf_precedente'] = df['cf'].shift(1)
df['test'] = (df['cf'] != df['cf_precedente'])
df = df[ df['test'] ]

#df.drop_duplicates(subset = 'id_impresa', inplace = True) 

df.sort_values(by='id_impresa', ascending=True, inplace=True)

cols_order = ['fonte', 'mm_aaaa','id_impresa', 'denominazione', 'cf', 'piva', 'prov', 'reg_imp_n',
        'sede_ul', 'n-albo_art', 'reg_imp_sez', 'ng2', 'stato_impresa',
       'data_cost', 'data_isc_ri', 'data_isc_rd', 'data_isc_aa', 'data_canc',
       'data_ini_at', 'data_cess_att',  'data_fall',
       'data_liquid', 'addetti_aaaa', 'addetti_indip', 'addetti_dip',
       'capitale', 'capitale_valuta', 'imp_sedi_ee', 'imp_eefvg', 'imp_pmi',
       'imp_startup', 'imp_femmilile', 'imp_giovanile', 'imp_straniera']


df[cols_order].to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False )



C:\Users\longato\AppData\Local\Temp\ipykernel_31420\3226212154.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(['cf', 'sede_ul' ], inplace = True)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\3226212154.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cf_precedente'] = df['cf'].shift(1)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\3226212154.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

# salvataggio file csv imprese e codici

In [37]:
file_risultati  = repo_dir + '\\' + 't_imprese_dp.csv'

cols_imp_dp =       [
                    'fonte', 'mm_aaaa', 'id_impresa', 
                    'cf','denominazione'
                    ]
df = df_anagrafica[cols_imp_dp]


df.drop_duplicates(subset = 'id_impresa', inplace = True) 

df.sort_values(by='id_impresa', ascending=True, inplace=True)
cols_order = ['fonte', 'mm_aaaa',  'id_impresa', 'cf', 'denominazione']
df[cols_order].to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False)


C:\Users\longato\AppData\Local\Temp\ipykernel_31420\930930524.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset = 'id_impresa', inplace = True)
C:\Users\longato\AppData\Local\Temp\ipykernel_31420\930930524.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='id_impresa', ascending=True, inplace=True)


In [38]:
file_risultati  = repo_dir + '\\' + 't_codici.csv'
cols_imp_dp = ['fonte', 'mm_aaaa', 'id_localiz', 'loc_n', 'ateco_tipo', 'ateco']
df = df_codici[cols_imp_dp]
df.sort_values(by='id_localiz', ascending=True, inplace=True)
df.to_csv(file_risultati, sep ='|',encoding='utf-8-sig', index=False)

C:\Users\longato\AppData\Local\Temp\ipykernel_31420\611961127.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='id_localiz', ascending=True, inplace=True)


In [39]:
print('end')

end
